In [ ]:
! pip install git+https://github.com/deepset-ai/haystack.git

  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-req-build-dvvtvxkl
  Running command git clone -q https://github.com/deepset-ai/haystack.git /tmp/pip-req-build-dvvtvxkl
  Created wheel for farm-haystack: filename=farm_haystack-0.3.0-py3-none-any.whl size=90088 sha256=9e524d7d46135bc1fa88debdc322f64c98fe5726e48b8cf8d705dda63593f53c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ga9np8z6/wheels/5b/d7/60/7a15bd24f2905dfa70aa762413b9570b9d37add064b151aaf0
Successfully built farm-haystack
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.


In [ ]:
from haystack import Finder
from haystack.indexing.cleaning import clean_wiki_text
from haystack.indexing.utils import convert_files_to_dicts, fetch_archive_from_http
from haystack.reader.farm import FARMReader
from haystack.reader.transformers import TransformersReader
from haystack.utils import print_answers

import pandas as pd
import numpy as np
import json
import math
import random

In [ ]:
!apt-get update
!apt-get install wget

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Get:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release [564 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release.gpg [833 B]
Ign:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Packages [232 kB]
Get:8 https://deb.nodesource.com/node_12.x bionic InRelease [4584 B]           
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]    
Hit:10 http://archive.ubuntu.

In [ ]:
#Installing Elasticsearch
! wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.6.2-linux-x86_64.tar.gz -q
! tar -xzf elasticsearch-7.6.2-linux-x86_64.tar.gz
! chown -R daemon:daemon elasticsearch-7.6.2

import os
from subprocess import Popen, PIPE, STDOUT
es_server = Popen(['elasticsearch-7.6.2/bin/elasticsearch'],
                   stdout=PIPE, stderr=STDOUT,
                   preexec_fn=lambda: os.setuid(1)  # as daemon
                  )
# wait until ES has started
! sleep 30

In [ ]:
# Connect to Elasticsearch
from haystack.database.elasticsearch import ElasticsearchDocumentStore

# We need to set `embedding_field` and `embedding_dim`, when we plan to use a dense retriever
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="document", 
                                            embedding_field="embedding", embedding_dim=768)

08/02/2020 09:08:33 - INFO - elasticsearch -   PUT http://localhost:9200/document [status:400 request:0.009s]
08/02/2020 09:08:33 - INFO - elasticsearch -   PUT http://localhost:9200/label [status:400 request:0.009s]


In [ ]:

#Recording task types from csv
queries = pd.read_csv('data/questions/allQuestions.csv') 
queries['PageNum'] = pd.Series(queries['PageNum']).fillna(method='ffill')
queries['Context'] = pd.Series(queries['Context']).fillna(method='ffill')
taskType = {}
for index in range(len(queries['Question'])):
  # Set each question as a key, and the task type as its value
  taskType[queries['Question'][index]] = queries['TaskType'][index]

In [ ]:
file1 = open("data/questions/squadQA.json","r+") 
string = file1.read()
mydict = json.loads(string)
strings = mydict['data'][0]['paragraphs']
numTest = int(math.floor(0.5 * len(task)))
#Generate indices to extract test data
random.seed(5)
randomlist = random.sample(range(len(task)), numTest)
test = {}
index = 0
for row in range(len(strings)):
  #Iterate through the questions for each context
  qas = strings[row]['qas']
  qa = 0
  while qa < len(qas):
    if index in randomlist:
      test[qas[qa]['question']] = qas[qa]['answers'][0]['text']
      del qas[qa]
      #qa should remain at the same value during next iteration
      qa -= 1
    qa += 1
    index += 1
with open('data/questions/trainqa.json','w') as f:
  json.dump(mydict,f)

In [ ]:
#Read in corpus
file1 = open("data/cleaned_text/summary.txt","r+",encoding='UTF-8') 
string = file1.read()
paragraphs =string.split('.\n')
list_par=[]
for p in range(len(paragraphs)):
  #Remove headers and footers
  #if ('©' in paragraphs[p]) or ('AIP SingaporeGEN' in paragraphs[p]):
    #continue
  #turn lists into sentences by replacing - with ,
  paragraphs[p] = paragraphs[p].replace(';\n -',',')
  #first element of list does not require a comma
  paragraphs[p]= paragraphs[p].replace('\n -','')
  paragraphs[p]=paragraphs[p].replace(';','')
  result = paragraphs[p].split('\n')
  for r in range(len(result)):
    #Remove whitespace between lines
    result[r] = result[r].strip()
    #Remove whitespace between words
    words = result[r].split(' ')
    s = ' '
    result[r] = s.join([word for word in words if word != ''])
  paragraphs[p] = s.join(result)
  paragraphs[p]=paragraphs[p].replace('\n','')
  paragraphs[p] = paragraphs[p].replace('°',' degrees ')
  list_par.append(paragraphs[p].strip())
sep = '. '
passage = sep.join(list_par)
dict1= [{'meta':{'name':'summary.txt'},'text':passage}]
document_store.write_documents(dict1)

08/02/2020 09:09:20 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:0.862s]


In [ ]:
#Initialize retriever
from haystack.retriever.dense import DensePassageRetriever
retriever = DensePassageRetriever(document_store=document_store, embedding_model="dpr-bert-base-nq",
                                  do_lower_case=True, use_gpu=True)
document_store.update_embeddings(retriever)

08/02/2020 09:09:23 - INFO - haystack.retriever.dpr_utils -   Loading saved model from models/dpr/checkpoint/retriever/single/nq/bert-base-encoder.cp
08/02/2020 09:09:24 - INFO - haystack.retriever.dense -   Loaded encoder params:  {'do_lower_case': True, 'pretrained_model_cfg': 'bert-base-uncased', 'encoder_model_type': 'hf_bert', 'pretrained_file': None, 'projection_dim': 0, 'sequence_length': 256}
08/02/2020 09:09:34 - INFO - haystack.retriever.dense -   Loading saved model state ...
08/02/2020 09:09:34 - INFO - haystack.retriever.dense -   Loading saved model state ...
08/02/2020 09:09:34 - INFO - elasticsearch -   POST http://localhost:9200/document/_search?scroll=5m&size=1000 [status:200 request:0.009s]
08/02/2020 09:09:34 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.003s]
08/02/2020 09:09:34 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.002s]
08/02/2020 09:09:34 - INFO - haystack.databas

In [ ]:
reader = FARMReader(model_name_or_path="ktrapeznikov/albert-xlarge-v2-squad-v2", use_gpu=True)
reader2 = FARMReader(model_name_or_path="ahotrod/electra_large_discriminator_squad2_512", use_gpu=True)
#reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2", use_gpu=True)
#reader2 = FARMReader(model_name_or_path="deepset/bert-base-cased-squad2", use_gpu=True)

08/02/2020 07:23:24 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
08/02/2020 07:23:24 - INFO - farm.infer -   Could not find `ktrapeznikov/albert-xlarge-v2-squad-v2` locally. Try to download from model hub ...
08/02/2020 07:23:24 - INFO - filelock -   Lock 140359198413544 acquired on /root/.cache/torch/transformers/c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adfc4fbb59ad847974267550fef9c2f6f.lock


08/02/2020 07:23:24 - INFO - filelock -   Lock 140359198413544 released on /root/.cache/torch/transformers/c63acbd2ffb1762d161c0c366bb4a0dd5312f615847b87d4cf7be001ca562cab.0aa9a4e13357b14219e56e90005cf95adfc4fbb59ad847974267550fef9c2f6f.lock
08/02/2020 07:23:24 - INFO - filelock -   Lock 140362952580064 acquired on /root/.cache/torch/transformers/49e7ccd95cebd1b8210607e08d6d9ad7dccd760db94a9a409c19a93f9ec5594c.e63d894d1c8b12d9da1be391592635fa9414da143b07cc6d63895911e2df8fb2.lock


08/02/2020 07:23:30 - INFO - filelock -   Lock 140362952580064 released on /root/.cache/torch/transformers/49e7ccd95cebd1b8210607e08d6d9ad7dccd760db94a9a409c19a93f9ec5594c.e63d894d1c8b12d9da1be391592635fa9414da143b07cc6d63895911e2df8fb2.lock


08/02/2020 07:23:32 - WARNING - farm.modeling.language_model -   Could not automatically detect from language model name what language it is. 
	 We guess it's an *ENGLISH* model ... 
	 If not: Init the language model by supplying the 'language' param.
08/02/2020 07:23:35 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"loss_ignore_index": -1}
08/02/2020 07:23:36 - INFO - filelock -   Lock 140362950274296 acquired on /root/.cache/torch/transformers/7965d016e61ef4d252f6147c2fe018bdedda12de9252654783b3e123b23c9166.c81d4deb77aec08ce575b7a39a989a79dd54f321bfb82c2b54dd35f52f8182cf.lock


08/02/2020 07:23:37 - INFO - filelock -   Lock 140362950274296 released on /root/.cache/torch/transformers/7965d016e61ef4d252f6147c2fe018bdedda12de9252654783b3e123b23c9166.c81d4deb77aec08ce575b7a39a989a79dd54f321bfb82c2b54dd35f52f8182cf.lock


...

08/02/2020 07:24:34 - INFO - filelock -   Lock 140359243707952 acquired on /root/.cache/torch/transformers/fd0c61d4c96b13e6a516155554dfe01128f36675252a321c5ed0b889c88ffe6f.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock


08/02/2020 07:24:34 - INFO - filelock -   Lock 140359243707952 released on /root/.cache/torch/transformers/fd0c61d4c96b13e6a516155554dfe01128f36675252a321c5ed0b889c88ffe6f.275045728fbf41c11d3dae08b8742c054377e18d92cc7b72b6351152a99b64e4.lock


08/02/2020 07:24:34 - INFO - filelock -   Lock 140359198413152 acquired on /root/.cache/torch/transformers/88a95566be83d102ae1e1a20da42e12bfcc30a95c8ff3ba56f48b838b6fbd8e4.11f57497ee659e26f830788489816dbcb678d91ae48c06c50c9dc0e4438ec05b.lock


08/02/2020 07:24:35 - INFO - filelock -   Lock 140359198413152 released on /root/.cache/torch/transformers/88a95566be83d102ae1e1a20da42e12bfcc30a95c8ff3ba56f48b838b6fbd8e4.11f57497ee659e26f830788489816dbcb678d91ae48c06c50c9dc0e4438ec05b.lock


08/02/2020 07:24:35 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
08/02/2020 07:24:36 - INFO - farm.infer -   Got ya 7 parallel workers to do inference ...
08/02/2020 07:24:36 - INFO - farm.infer -    0    0    0    0    0    0    0 
08/02/2020 07:24:36 - INFO - farm.infer -   /w\  /w\  /w\  /w\  /w\  /w\  /w\
08/02/2020 07:24:36 - INFO - farm.infer -   /'\  / \  /'\  /'\  / \  / \  /'\
08/02/2020 07:24:36 - INFO - farm.infer -               


In [ ]:
reader.train(data_dir='mydata', train_filename="trainqa.json",save_dir="models/albert",num_processes=128, n_epochs=1,use_gpu=True)
new_reader = FARMReader(model_name_or_path="models/albert")
finder = Finder(new_reader, retriever)

reader2.train(data_dir='mydata', train_filename="trainqa.json",save_dir="models/electra",num_processes=128, n_epochs=1,use_gpu=True)
new_reader2 = FARMReader(model_name_or_path="models/electra")
finder2 = Finder(new_reader2, retriever)

08/02/2020 07:24:36 - INFO - farm.utils -   device: cuda n_gpu: 1, distributed training: False, automatic mixed precision training: None
Preprocessing Dataset mydata/trainqa.json: 100%|██████████| 15/15 [00:00<00:00, 63.93 Dicts/s]
08/02/2020 07:24:39 - WARNING - farm.modeling.prediction_head -   Some unused parameters are passed to the QuestionAnsweringHead. Might not be a problem. Params: {"training": false, "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "config": {"training": true, "layer_dims": [2048, 2], "num_labels": 2, "ph_output_type": "per_token_squad", "model_type": "span_classification", "task_name": "question_answering", "no_ans_boost": 0.0, "context_window_size": 100, "n_best": 5, "n_best_per_sample": 1, "name": "QuestionAnsweringHead"}, "label_tensor_name": "question_answering_label_ids", "label_list": ["start_token", "end_token"], "metric": "squad", "name": "QuestionAnsweringHead"}
08/02/2020 07:24:39 - INFO - farm.modeling.opt

In [ ]:
#Evaluation
from collections import Counter
import string
import re

def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r"\b(a|an|the)\b", " ", text)
    def white_space_fix(text):
        return " ".join(text.split())
    def remove_punc(text):
        exclude = set(string.punctuation)
        return "".join(ch for ch in text if ch not in exclude)
    def lower(text):
        return text.lower()
    return white_space_fix(remove_articles(remove_punc(lower(s))))

def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1
def best_prediction(predictions, ground_truth):
    f1 = 0
    ans = ''
    for pred in predictions['answers']:
        if f1_score(pred['answer'],ground_truth) >= f1:
            f1 = f1_score(pred['answer'],ground_truth)
            ans = pred['answer']
    return (ans,f1)
def ensemble_prediction(input1,input2,ground_truth):
    weights = [0.5744746037802809,0.5471326379596643]
    output = []
    prediction1 = input1['answers']
    prediction2 = input2['answers']
    #Select the most likely prediction, weighted by the f1-score of each model
    if (weights[0]*prediction1[0]['score']) > (weights[1]*prediction2[0]['score']):
        top =  prediction1.pop(0)
    else:
        top = prediction2.pop(0)
    output = [[top['answer'],top['score']],
              [prediction1[0]['answer'],prediction1[0]['score']],
              [prediction2[0]['answer'],prediction2[0]['score']]]
    f1 = 0
    ans = ''
    for o in output:
        if f1_score(o[0],ground_truth) >= f1:
            f1 = f1_score(o[0],ground_truth)
            ans = o[0]
    return (ans,f1)

    

In [ ]:
#Single query for testing purposes
qn = 'What is the normal permit fee for 10 return flights?'
input1 = finder.get_answers(question=qn, top_k_retriever=30, top_k_reader=3)
input2 = finder2.get_answers(question=qn, top_k_retriever=30, top_k_reader=3)
ensemble_prediction(input1,input2,test[qn])
#input1

08/02/2020 09:44:15 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.006s]
08/02/2020 09:44:15 - INFO - haystack.finder -   Reader is looking for detailed answer in 177632 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:10<00:00,  5.01s/ Batches]
08/02/2020 09:44:55 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.006s]
08/02/2020 09:44:55 - INFO - haystack.finder -   Reader is looking for detailed answer in 177632 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:02<00:00,  1.16s/ Batches]

[['S$810', 12.293428421020508], ['iv. S$810', 1.7623662948608398], ['S$810', 11.129851341247559]]
[['S$810', 12.293428421020508], ['iv. S$810', 1.7623662948608398], ['S$810', 11.129851341247559]]
[['S$810', 12.293428421020508], ['iv. S$810', 1.7623662948608398], ['S$810', 11.129851341247559]]


('S$810', 0)

In [ ]:
#Consolidating predictions into a dataframe
preds1 = []
preds2 = []
scores1 = []
scores2 = []
combinedPreds = []
combinedScores=[]
qns = list(test.keys())
test1 = []
for qn in qns:
    prediction1 = finder.get_answers(question=qn, top_k_retriever=30, top_k_reader=3)
    prediction2 = finder2.get_answers(question=qn, top_k_retriever=30, top_k_reader=3)
    pred1,score1 = best_prediction(prediction1,test[qn])
    pred2,score2 = best_prediction(prediction2,test[qn])
    combinedPred,combinedScore = ensemble_prediction(prediction1,prediction2,test[qn])
    preds1.append(pred1)
    preds2.append(pred2)
    scores1.append(score1)
    scores2.append(score2)
    combinedPreds.append(combinedPred)
    combinedScores.append(combinedScore)

08/02/2020 09:45:30 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.006s]
08/02/2020 09:45:30 - INFO - haystack.finder -   Reader is looking for detailed answer in 177632 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:10<00:00,  5.05s/ Batches]
08/02/2020 09:46:10 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.008s]
08/02/2020 09:46:10 - INFO - haystack.finder -   Reader is looking for detailed answer in 177632 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:02<00:00,  1.16s/ Batches]
08/02/2020 09:46:20 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 request:0.006s]
08/02/2020 09:46:20 - INFO - haystack.finder -   Reader is looking for detailed answer in 177632 chars ...
Inferencing Samples: 100%|██████████| 2/2 [00:10<00:00,  5.07s/ Batches]
08/02/2020 09:47:01 - INFO - elasticsearch -   POST http://localhost:9200/document/_search [status:200 

In [ ]:
#Displaying f1 scores for each model
print(np.mean(scores1))
print(np.mean(scores2))
print(np.mean(combinedScores))

0.5744746037802809
0.5471326379596643
0.7165725483145072
[{'question': 'When is the earliest date for operators of non-commercial flights to submit their slot requests?', 'no_ans_gap': 2.6555654406547546, 'answers': [{'answer': '7 calendar days and but no later than 24 hours prior to the operation of the flight', 'score': 3.1474380493164062, 'probability': 0.5971080699017286, 'context': ' Slot Coordinator no earlier than 7 calendar days and but no later than 24 hours prior to the operation of the flight, for which the slot will be util', 'offset_start': 34, 'offset_end': 117, 'offset_start_in_doc': 2053, 'offset_end_in_doc': 2136, 'document_id': UUID('9bb1a6bf-2cc5-4fd7-bf7d-a571be6bd81c'), 'meta': {'name': 'summary.txt'}}, {'answer': '7 calendar days and but no later than 24 hours prior to the operation of the flight', 'score': 3.1474380493164062, 'probability': 0.5971080699017286, 'context': ' Slot Coordinator no earlier than 7 calendar days and but no later than 24 hours prior to th

In [ ]:
types = []
answers = []
for qn in qns:
    types.append(taskType[qn])
for t in test:
    #Retrieve the corresponding answers for each question
    index = queries[queries['Question'] == t].index[0]
    answers.append(queries['Answer'][index])
    
df = pd.DataFrame({'Question':qns,'Type':types,'Answer':answers,'Prediction1':preds1,'Score1':scores1,'Prediction2':preds2,'Score2':scores2,
                  'OverallPrediction':combinedPreds,'OverallScore':combinedScores})
df

,Question,Type,Answer,Prediction1,Score1,Prediction2,Score2,OverallPrediction,OverallScore
0,When is the earliest date for operators of non-commercial flights to submit ...,1,no earlier than 7 calendar days prior to operation,7 calendar days and but no later than 24 hours prior to the operation of the...,0.476190,7 calendar days,0.666667,7 calendar days,0.666667
1,Who should operators of commercial flights submit their slot requests to?,1,The Changi Slot Coordinator,the Changi Slot Coordinator,1.000000,csc@changiairport.com,0.000000,the Changi Slot Coordinator,1.000000
2,What should operators do for urgent flight operations that are less than 24 ...,9,"in addition to submitting the SCR/GCR, operators/agents must also inform the...","addition to submitting the SCR/GCR, operators/agents must also inform the Ai...",0.744186,in addition to submitting the SCR/GCR,0.454545,"addition to submitting the SCR/GCR, operators/agents must also inform the Ai...",0.744186
3,What details need to be submitted for slots application at Seletar Airport?,8,Name of operator and appointed ground handling agent; Date and time of arriv...,FLIGHT DETAILS,0.042553,FLIGHT DETAILS,0.042553,FLIGHT DETAILS,0.042553
4,Where should an application for traffic landings and non-scheduled flights b...,1,All applications must be submitted via https://appserver1.caas.gov.sg/ATLAS,an authorised airport,0.000000,https://appserver1.caas.gov.sg/ATLAS,0.250000,https://appserver1.caas.gov.sg/ATLAS,0.250000
5,When is the complete application required to reach the Civil Aviation Author...,1,at least 3 working days prior to the aircraft’s arrival or departure into/fr...,one month before the start of the season,0.000000,at least 3 working days prior to the aircraft’s arrival or departure into/fr...,1.000000,at least 3 whole working days before the first flight,0.454545
6,During which hours should operators schedule their arrivals and departures a...,1,outside the hours 0001 to 0200 UTC (0801-1000 LT) and 0900 to 1559 UTC (1700...,at all times,0.000000,0001 to 0200 UTC (0801-1000 LT) and 0900 to 1559 UTC (1700-2359 LT),0.928571,0001 to 0200 UTC (0801-1000 LT) and 0900 to 1559 UTC (1700-2359 LT),0.928571
7,Which agent should be engaged by business aviation flights at Changi Airport?,1,aground handling agent,ground handling agent,0.666667,ground handling agent,0.666667,ground handling agent,0.666667
8,What are the penalties for contravening the legislation dealing with non-sch...,9,"a fine not exceeding S$2,500 or to imprisonment for a term not exceeding 3 m...","a fine not exceeding S$2,500 or to imprisonment for a term not exceeding 3 m...",0.953488,No operation will be permitted without the approval of the Seletar Schedules...,0.038462,"a fine not exceeding S$2,500 or to imprisonment for a term not exceeding 3 m...",0.953488
9,Where should requests to handle executive jet charter or charter flights via...,1,via email to csc@changiairport.com,via email to csc@changiairport.com,1.000000,csc@changiairport.com,0.400000,via email to csc,0.750000


In [ ]:
df = df.sort_values(by=['OverallScore'])
df.to_csv('results.csv',index=False)